Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [165]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import math
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [166]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (51909, 28, 28) (51909,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [167]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (51909, 784) (51909, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [168]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [206]:
class Model(object):
    def __init__(self, image_size=28, num_hidden=[1024], num_labels=10, 
                 l2_weight = 0.0, keep_prob = 1.0, 
                 learning_rate = 0.5, momentum = 0.5, 
                 decay_steps = 10000, decay_rate = 1.0):

        self.image_size = image_size
        self.num_hidden = num_hidden
        self.num_labels = num_labels
        self.l2_weight = l2_weight
        self.keep_prob = keep_prob
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.momentum = momentum
        
        
        # Initialize weights and biases
        self.weights = []
        self.biases = []
        
        # Create a Graph
        self.graph = tf.Graph()

        # Build the graph
        with self.graph.as_default():

            # Input data
            self.tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
            self.tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
            self.tf_keep_prob = tf.placeholder(tf.float32)
            
            # Epsilon
            epsilon = tf.constant(value=0.00001)

            # Variables
            self.init_variables()

            # Inference
            self.logits = self.inference()

            # Loss
            total_loss = tf.nn.softmax_cross_entropy_with_logits(
                labels=self.tf_labels, logits=(self.logits+epsilon))
            
            for w in self.weights:
                total_loss += + l2_weight*tf.nn.l2_loss(w) 
            
            self.loss = tf.reduce_mean(total_loss)
            
            # Step Counter
            self.tf_global_step = tf.Variable(0)
            
            # Adaptive learning rate
            self.tf_learning_rate = tf.train.exponential_decay(self.learning_rate, self.tf_global_step, 
                                                               self.decay_steps, self.decay_rate)

            # Optimizer.
            #self.optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(self.loss)
            self.optimizer = tf.train.MomentumOptimizer(self.learning_rate, self.momentum).minimize(self.loss, 
                                                                                  global_step=self.tf_global_step)

            # Predictions for the training, validation, and test data.
            self.prediction = tf.nn.softmax(self.logits)

    
    
    def init_variables(self):

        # Initialize weight and biases lists to empty lists
        self.weights = []
        self.biases = []

        # Input size for each layer, initialize to the size of input images
        layer_input_size = self.image_size * self.image_size

        # Initialize weights and biases for each layer
        for n in self.num_hidden + [self.num_labels]:
            w = tf.Variable(tf.truncated_normal([layer_input_size, n], 
                                                stddev=1.0 / math.sqrt(float(layer_input_size))))
            b = tf.Variable(tf.zeros([n]))
            
            self.weights.append(w)
            self.biases.append(b)

            # Update input size for the next layer
            layer_input_size = n
 
    def inference(self):

        # Input for the layer, initialize to the input dataset
        layer_input = self.tf_dataset
        layer_logits = None

        for w, b in zip(self.weights, self.biases):
            if layer_logits is not None:
                layer_input = tf.nn.dropout(tf.nn.relu(layer_logits), self.tf_keep_prob)  
            layer_logits = tf.matmul(layer_input, w) + b

        return layer_logits




    def train(self, train_dataset, train_labels, valid_dataset, valid_labels, test_dataset, test_labels,
              num_steps=1000, batch_size=128):

        with tf.Session(graph=self.graph) as session:

            # initialize variables
            tf.global_variables_initializer().run()
            print("Variables Initialized")

            # Initial values for accuracies
            train_accuracy = 0.0
            valid_accuracy = 0.0
            test_accuracy = 0.0
            
            print("Training for %d steps" % num_steps)
            for step in range(num_steps):

                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

                # Generate a minibatch.
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]

                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {self.tf_dataset : batch_data, self.tf_labels : batch_labels,
                            self.tf_keep_prob : self.keep_prob}
                session.run([self.optimizer, self.loss, self.prediction], feed_dict=feed_dict)

                if (step % 500 == 0):
                    
                    
                    feed_dict = {self.tf_dataset : batch_data, self.tf_labels : batch_labels,
                            self.tf_keep_prob : 1.0}
                    _, lr, l, predictions = session.run([self.optimizer, self.tf_learning_rate,
                                                         self.loss, self.prediction], 
                                                feed_dict=feed_dict)
                    print("Minibatch loss and learning rate at step %d: %f / %f" % (step, l, lr))
                    train_accuracy = accuracy(predictions, batch_labels)
                    
                    feed_dict = {self.tf_dataset : valid_dataset, self.tf_labels : valid_labels,
                            self.tf_keep_prob : 1.0}
                    _, l, predictions = session.run([self.optimizer, self.loss, self.prediction], 
                                                feed_dict=feed_dict)
                    valid_accuracy = accuracy(predictions, valid_labels)
                    print("Training & validation accuracy: %.1f%% / %.1f%%" % (train_accuracy, valid_accuracy))
                    
            
            feed_dict = {self.tf_dataset : test_dataset, self.tf_labels : test_labels,
                    self.tf_keep_prob : 1.0}
            _, l, predictions = session.run([self.optimizer, self.loss, self.prediction], 
                                        feed_dict=feed_dict)
                    
            test_accuracy = accuracy(predictions, test_labels)
            print("Test accuracy: %.1f%%" % test_accuracy)   
            
        return train_accuracy, valid_accuracy, test_accuracy

    

In [211]:



# Build the model with batch size of 128, one hidden layer with 1024 units 
batch_size = 128
num_steps = 6001
num_hidden = [1024]
model = Model(image_size=image_size, num_hidden=num_hidden, num_labels=num_labels,
              l2_weight = 0.001)
train_accuracy, valid_accuracy, test_accuracy = model.train(
    train_dataset, train_labels, 
    valid_dataset, valid_labels, 
    test_dataset, test_labels, 
    num_steps=num_steps, batch_size=batch_size)


Variables Initialized
Training for 6001 steps
Minibatch loss and learning rate at step 0: 1.832280 / 0.500000
Training & validation accuracy: 55.5% / 58.8%
Minibatch loss and learning rate at step 500: 0.489812 / 0.500000
Training & validation accuracy: 91.4% / 83.2%
Minibatch loss and learning rate at step 1000: 0.353023 / 0.500000
Training & validation accuracy: 95.3% / 86.1%
Minibatch loss and learning rate at step 1500: 0.332405 / 0.500000
Training & validation accuracy: 96.1% / 86.7%
Minibatch loss and learning rate at step 2000: 0.327295 / 0.500000
Training & validation accuracy: 96.1% / 84.7%
Minibatch loss and learning rate at step 2500: 0.286901 / 0.500000
Training & validation accuracy: 96.9% / 85.2%
Minibatch loss and learning rate at step 3000: 0.275459 / 0.500000
Training & validation accuracy: 99.2% / 86.7%
Minibatch loss and learning rate at step 3500: 0.318491 / 0.500000
Training & validation accuracy: 97.7% / 86.0%
Minibatch loss and learning rate at step 4000: 0.35428

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [212]:
small_size = 256
small_train_dataset = train_dataset[:small_size] 
small_train_labels = train_labels[:small_size]
print('Training set', small_train_dataset.shape, small_train_labels.shape)

model = Model(image_size=image_size, num_hidden=num_hidden, num_labels=num_labels,
              l2_weight = 0.0)
# Build the model with batch size of 128, one hidden layer with 1024 units 
train_accuracy, valid_accuracy, test_accuracy = model.train(
    small_train_dataset, small_train_labels, 
    valid_dataset, valid_labels, 
    test_dataset, test_labels, 
    num_steps=num_steps, batch_size=batch_size)


Training set (256, 784) (256, 10)
Variables Initialized
Training for 6001 steps
Minibatch loss and learning rate at step 0: 1.438153 / 0.500000
Training & validation accuracy: 56.2% / 59.2%
Minibatch loss and learning rate at step 500: 0.000421 / 0.500000
Training & validation accuracy: 100.0% / 72.1%
Minibatch loss and learning rate at step 1000: 0.000206 / 0.500000
Training & validation accuracy: 100.0% / 75.1%
Minibatch loss and learning rate at step 1500: 0.000147 / 0.500000
Training & validation accuracy: 100.0% / 76.8%
Minibatch loss and learning rate at step 2000: 0.000121 / 0.500000
Training & validation accuracy: 100.0% / 78.0%
Minibatch loss and learning rate at step 2500: 0.000106 / 0.500000
Training & validation accuracy: 100.0% / 78.8%
Minibatch loss and learning rate at step 3000: 0.000097 / 0.500000
Training & validation accuracy: 100.0% / 79.5%
Minibatch loss and learning rate at step 3500: 0.000091 / 0.500000
Training & validation accuracy: 100.0% / 80.0%
Minibatch los

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [213]:

# Build the model with batch size of 128, one hidden layer with 1024 units 
model = Model(image_size=image_size, num_hidden=num_hidden, num_labels=num_labels,
              l2_weight = 0.0, keep_prob = 0.2)
train_accuracy, valid_accuracy, test_accuracy = model.train(
    small_train_dataset, small_train_labels, 
    valid_dataset, valid_labels, 
    test_dataset, test_labels, 
    num_steps=num_steps, batch_size=batch_size)

Variables Initialized
Training for 6001 steps
Minibatch loss and learning rate at step 0: 1.483992 / 0.500000
Training & validation accuracy: 55.5% / 56.5%
Minibatch loss and learning rate at step 500: 0.000016 / 0.500000
Training & validation accuracy: 100.0% / 72.7%
Minibatch loss and learning rate at step 1000: 0.000007 / 0.500000
Training & validation accuracy: 100.0% / 74.9%
Minibatch loss and learning rate at step 1500: 0.000005 / 0.500000
Training & validation accuracy: 100.0% / 76.4%
Minibatch loss and learning rate at step 2000: 0.000005 / 0.500000
Training & validation accuracy: 100.0% / 77.7%
Minibatch loss and learning rate at step 2500: 0.000004 / 0.500000
Training & validation accuracy: 100.0% / 78.4%
Minibatch loss and learning rate at step 3000: 0.000004 / 0.500000
Training & validation accuracy: 100.0% / 78.9%
Minibatch loss and learning rate at step 3500: 0.000004 / 0.500000
Training & validation accuracy: 100.0% / 79.5%
Minibatch loss and learning rate at step 4000: 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [221]:
batch_size = 256
num_steps = 100001
num_hidden = [1024, 128]
model = Model(image_size=image_size, num_hidden=num_hidden, num_labels=num_labels,
              l2_weight = 0.001, keep_prob = 0.5, learning_rate = 0.4, momentum = 0.5,
              decay_steps = 10000, decay_rate = 0.9)
train_accuracy, valid_accuracy, test_accuracy = model.train(
    train_dataset, train_labels, 
    valid_dataset, valid_labels, 
    test_dataset, test_labels, 
    num_steps=num_steps, batch_size=batch_size)


Variables Initialized
Training for 100001 steps
Minibatch loss and learning rate at step 0: 2.557856 / 0.399996
Training & validation accuracy: 52.3% / 49.7%
Minibatch loss and learning rate at step 500: 0.673785 / 0.397886
Training & validation accuracy: 88.3% / 85.1%
Minibatch loss and learning rate at step 1000: 0.560609 / 0.395787
Training & validation accuracy: 88.7% / 86.2%
Minibatch loss and learning rate at step 1500: 0.515693 / 0.393699
Training & validation accuracy: 88.7% / 87.5%
Minibatch loss and learning rate at step 2000: 0.447221 / 0.391622
Training & validation accuracy: 90.2% / 87.4%
Minibatch loss and learning rate at step 2500: 0.449572 / 0.389556
Training & validation accuracy: 92.2% / 86.9%
Minibatch loss and learning rate at step 3000: 0.459169 / 0.387501
Training & validation accuracy: 91.8% / 87.3%
Minibatch loss and learning rate at step 3500: 0.444807 / 0.385457
Training & validation accuracy: 91.4% / 87.6%
Minibatch loss and learning rate at step 4000: 0.382